imports

In [2]:
from sklearn.datasets import make_classification, make_regression
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


data simulation

In [3]:
#for sober/intoxicated classifier

features, output = make_classification(n_samples = 1000,
                                       n_features = 3,
                                       n_informative= 3,
                                       n_redundant=0,
                                       n_classes = 2,
                                       weights = [0.75, 0.25])

#weights are for class generation. Roughly expect to play at baseline 90% of the time and deviated the other 10%

print(pd.DataFrame(features, columns=['Reaction Time', 'Accuracy', 'Literally Anything Else']).head())
print(pd.DataFrame(output, columns=['Class']))



   Reaction Time  Accuracy  Literally Anything Else
0      -0.150067 -0.425658                 0.434978
1       1.597779 -1.070777                 0.256357
2       2.089832  2.160973                -2.375608
3      -1.224791 -1.329641                 0.918254
4      -0.480642  0.417487                -0.725835
     Class
0        1
1        0
2        1
3        0
4        1
..     ...
995      0
996      0
997      0
998      0
999      0

[1000 rows x 1 columns]


In [4]:
#mlp

class MLP(nn.Module) :
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3, 9, bias=True)
        self.h1 = nn.Linear(9, 3, bias=True)
        self.h2 = nn.Linear(3, 2, bias=True)

    def forward(self, x):
        x = torch.sigmoid(self.l1(x))
        x = torch.sigmoid(self.h1(x))
        x = torch.sigmoid(self.h2(x))
        return x

In [7]:
#!pip install skorch
from skorch import NeuralNetClassifier

features = features.astype(np.float32)
output = output.astype(np.int64)

trainFraction = .8
sample = np.random.uniform(size = 1000) < trainFraction
xtrain = features[sample]
ytrain = output[sample]
xtest = features[~sample]
ytest = output[~sample]

#ytrain = torch.tensor(ytrain).float()
#ytest = torch.tensor(ytest).float()

xtorch = torch.from_numpy(xtrain).float()
ytorch = torch.from_numpy(ytrain).float().unsqueeze(1)

learning_rate = 1.25
model = MLP()

def init_weights(m):
  if isinstance(m, nn.Linear):
    torch.nn.init.xavier_uniform(m.weight)
    m.bias.data.fill_(0.01)

model.apply(init_weights)

net = NeuralNetClassifier(model, max_epochs = 100, lr = learning_rate, iterator_train__shuffle = True)
net.fit(xtrain, ytrain)

preds = net.predict(xtest)
acc = 0
tot = 0
for i in range(len(preds)) :
  if preds[i] == ytest[i] :
    acc += 1
    tot += 1
  else :
    tot += 1
print("MLP Test Accuracy:", acc/tot)


ModuleNotFoundError: No module named 'skorch'

In [93]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()

logReg.fit(xtrain, ytrain)
print("Logistic Regression Test Accuracy:", logReg.score(xtest, ytest))

Logistic Regression Test Accuracy: 0.775609756097561
